In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F  # Import F for functional operations

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(self._get_conv_output((3, 32, 32)), 120)  # Updated
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def _get_conv_output(self, shape):
        # Create a dummy input to pass through the network
        with torch.no_grad():
            return self._forward(torch.autograd.Variable(torch.ones(1, *shape))).view(1, -1).size(1)

    def _forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        return x

    def forward(self, x):
        x = self._forward(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor dynamically
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load and preprocess data
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

# Initialize and train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available
model = SimpleCNN().to(device)  # Move model to device
criterion = nn.CrossEntropyLoss()  # Define loss function
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Define optimizer

for epoch in range(10):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to device

        optimizer.zero_grad()  # Zero the gradients

        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()
        if i % 2000 == 1999:    # Print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation
    for data in testloader:
        images, labels = data
        outputs = model(images.to(device))  # Forward pass
        _, predicted = torch.max(outputs.data, 1)  # Get predicted labels
        total += labels.size(0)
        correct += (predicted.cpu() == labels).sum().item()  # Count correct predictions

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')


Files already downloaded and verified
Files already downloaded and verified
[1, 2000] loss: 2.163
[1, 4000] loss: 1.741
[1, 6000] loss: 1.559
[1, 8000] loss: 1.426
[1, 10000] loss: 1.355
[1, 12000] loss: 1.293
[2, 2000] loss: 1.186
[2, 4000] loss: 1.162
[2, 6000] loss: 1.117
[2, 8000] loss: 1.108
[2, 10000] loss: 1.057
[2, 12000] loss: 1.060
[3, 2000] loss: 0.949
[3, 4000] loss: 0.940
[3, 6000] loss: 0.923
[3, 8000] loss: 0.915
[3, 10000] loss: 0.919
[3, 12000] loss: 0.915
[4, 2000] loss: 0.779
[4, 4000] loss: 0.815
[4, 6000] loss: 0.812
[4, 8000] loss: 0.811
[4, 10000] loss: 0.811
[4, 12000] loss: 0.824
[5, 2000] loss: 0.670
[5, 4000] loss: 0.699
[5, 6000] loss: 0.706
[5, 8000] loss: 0.688
[5, 10000] loss: 0.717
[5, 12000] loss: 0.733
[6, 2000] loss: 0.587
[6, 4000] loss: 0.603
[6, 6000] loss: 0.603
[6, 8000] loss: 0.621
[6, 10000] loss: 0.641
[6, 12000] loss: 0.618
[7, 2000] loss: 0.482
[7, 4000] loss: 0.504
[7, 6000] loss: 0.534
[7, 8000] loss: 0.541
[7, 10000] loss: 0.576
[7, 12000